# Lab 6

Scikit learn provides a large variety of algorithms for some common Machine Learning tasks, such as:

* Classification
* Regression
* Clustering
* Feature Selection
* Anomaly Detection

It also provides some datasets that you can use to test these algorithms:

* Classification Datasets:
    * Breast cancer wisconsin
    * Iris plants (3-classes)
    * Optical recognition of handwritten digits (10-classes)
    * Wine (n-classes)

* Regression Datasets: 
    * Boston house prices 
    * Diabetes
    * Linnerrud (multiple regression)
    * California Housing

* Image: 
    * The Olivetti faces
    * The Labeled Faces in the Wild face recognition
    * Forest covertypes

* NLP:
    * News group
    * Reuters Corpus Volume I 

* Other:
    * Kddcup 99- Intrusion Detection

## Exercises

1. Use the full [Kddcup](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html) dataset to compare classification performance of 3 different classifiers. 
    * Separate the data into train, validation, and test. 
    * Use accuracy as the metric for assessing performance. 
    * For each classifier, identify the hyperparameters. Perform optimization over at least 2 hyperparameters.   
    * Compare the performance of the optimal configuration of the classifiers.

## Quick look at the data

In [1]:
import pandas as pd
from sklearn.datasets import fetch_kddcup99
from sklearn.preprocessing import OneHotEncoder
D = fetch_kddcup99(as_frame=True) 


In [2]:
df = pd.DataFrame(D.data)

df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,b'tcp',b'http',b'SF',181,5450,0,0,0,0,...,9,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0
1,0,b'tcp',b'http',b'SF',239,486,0,0,0,0,...,19,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0
2,0,b'tcp',b'http',b'SF',235,1337,0,0,0,0,...,29,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
3,0,b'tcp',b'http',b'SF',219,1337,0,0,0,0,...,39,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
4,0,b'tcp',b'http',b'SF',217,2032,0,0,0,0,...,49,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0


In [3]:
df.iloc[:,[1,2,3]] #non numerical columns-- model fit throwing error
df['protocol_type'].unique()
#df['service'].unique()
#df['flag'].unique()

array([b'tcp', b'udp', b'icmp'], dtype=object)

In [4]:
pt_enc = pd.get_dummies(df['protocol_type'], prefix='ptype', dtype = 'int')
pt_enc.head()
df = pd.concat([df, pt_enc], axis=1)
df.drop('protocol_type', axis=1, inplace=True)
df.head()

,duration,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,...,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,ptype_b'icmp',ptype_b'tcp',ptype_b'udp'
0,0,b'http',b'SF',181,5450,0,0,0,0,0,...,0.0,0.11,0.0,0.0,0.0,0.0,0.0,0,1,0
1,0,b'http',b'SF',239,486,0,0,0,0,0,...,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0,1,0
2,0,b'http',b'SF',235,1337,0,0,0,0,0,...,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0,1,0
3,0,b'http',b'SF',219,1337,0,0,0,0,0,...,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0,1,0
4,0,b'http',b'SF',217,2032,0,0,0,0,0,...,0.0,0.02,0.0,0.0,0.0,0.0,0.0,0,1,0


In [5]:
s_enc = pd.get_dummies(df['service'], prefix='service', dtype = 'int')
s_enc.head()
df = pd.concat([df, s_enc], axis=1)
df.drop('service', axis=1, inplace=True)
df.head()

,duration,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,...,service_b'telnet',service_b'tftp_u',service_b'tim_i',service_b'time',service_b'urh_i',service_b'urp_i',service_b'uucp',service_b'uucp_path',service_b'vmnet',service_b'whois'
0,0,b'SF',181,5450,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,b'SF',239,486,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,b'SF',235,1337,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,b'SF',219,1337,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,b'SF',217,2032,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [6]:
f_enc = pd.get_dummies(df['flag'], prefix='flag', dtype = 'int')
f_enc.head()
df = pd.concat([df, f_enc], axis=1)
df.drop('flag', axis=1, inplace=True)
df.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_b'REJ',flag_b'RSTO',flag_b'RSTOS0',flag_b'RSTR',flag_b'S0',flag_b'S1',flag_b'S2',flag_b'S3',flag_b'SF',flag_b'SH'
0,0,181,5450,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,0,239,486,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,0,235,1337,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,0,219,1337,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,0,217,2032,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


fixed that

## Exercise 1: 
- Use the full Kddcup dataset to compare classification performance of 3 different classifiers.
- Separate the data into train, validation, and test.
- Use accuracy as the metric for assessing performance.
- For each classifier, identify the hyperparameters. Perform optimization over at least 2 hyperparameters.
- Compare the performance of the optimal configuration of the classifiers.

### Preprocessing

In [7]:
#I'll try decision tree, knn, and neural net
from sklearn.model_selection import train_test_split
from sklearn import preprocessing 

from sklearn.neighbors import KNeighborsClassifier

In [8]:
df['target'] = D.target

In [9]:
target_label_encoder = preprocessing.LabelEncoder() 

df['target']= target_label_encoder.fit_transform(df['target']) 
  
df['target'].unique() 

array([11,  1,  7, 12,  9, 18,  3, 14, 20, 15,  5,  6,  2,  0,  4, 17, 13,
       10,  8, 22, 21, 19, 16])

In [10]:
df = df.astype('float32')
df.dtypes

duration          float32
src_bytes         float32
dst_bytes         float32
land              float32
wrong_fragment    float32
                   ...   
flag_b'S2'        float32
flag_b'S3'        float32
flag_b'SF'        float32
flag_b'SH'        float32
target            float32
Length: 119, dtype: object

In [11]:
X = df.drop('target', axis=1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

### KNN Tuning and Fit

In [18]:
neigh = KNeighborsClassifier()

param_grid = {
    'n_neighbors': [3, 7],
    'weights': ['uniform'],
    'leaf_size': [20, 30]
}

#getting error that classes are less than n_splits, so cv has to be 2, dont want to eliminate the smaller class yet
g_search = GridSearchCV(neigh, param_grid, cv=2, scoring='accuracy', n_jobs = 1, verbose = 2)
g_search.fit(X_train, y_train)

print("Best Parameters:", g_search.best_params_)
print("Best Score:", g_search.best_score_)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] END .......leaf_size=20, n_neighbors=3, weights=uniform; total time= 1.1min
[CV] END .......leaf_size=20, n_neighbors=3, weights=uniform; total time= 1.1min
[CV] END .......leaf_size=20, n_neighbors=7, weights=uniform; total time= 1.1min
[CV] END .......leaf_size=20, n_neighbors=7, weights=uniform; total time= 1.1min
[CV] END .......leaf_size=30, n_neighbors=3, weights=uniform; total time= 1.1min
[CV] END .......leaf_size=30, n_neighbors=3, weights=uniform; total time= 1.1min
[CV] END .......leaf_size=30, n_neighbors=7, weights=uniform; total time= 1.1min
[CV] END .......leaf_size=30, n_neighbors=7, weights=uniform; total time= 1.1min
Best Parameters: {'leaf_size': 20, 'n_neighbors': 3, 'weights': 'uniform'}
Best Score: 0.9982364074328975


In [19]:
#best_knn = random_search.best_estimator_
#y_pred = best_knn.predict(X_test)

print('Best knn:', best_knn)
print('best knn prediction:', y_pred)

Best knn: KNeighborsClassifier(leaf_size=20, n_neighbors=3)
best knn prediction: [18. 18. 18. ... 11. 18. 11.]


### SVM

In [13]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

#svm works best scaled
pipeline = Pipeline([('scaler', StandardScaler()),('svm', SVC())])

param_grid = {'svm__C': [0.1, 5], 'svm__kernel': ['linear'], 'svm__gamma' : [.05, 5, 25]} #rbf and poly were taking days

#getting error that classes are less than n_splits, so cv has to be 2, dont want to eliminate the smaller class yet

g_search = GridSearchCV(pipeline, param_grid, cv=2, scoring='accuracy', n_jobs = 1, verbose =2)
g_search.fit(X_train, y_train)

print("Best Parameters:", g_search.best_params_)
print("Best Score:", g_search.best_score_)

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV] END ....svm__C=0.1, svm__gamma=0.05, svm__kernel=linear; total time=  19.2s
[CV] END ....svm__C=0.1, svm__gamma=0.05, svm__kernel=linear; total time=  16.2s
[CV] END .......svm__C=0.1, svm__gamma=5, svm__kernel=linear; total time=  18.9s
[CV] END .......svm__C=0.1, svm__gamma=5, svm__kernel=linear; total time=  16.2s
[CV] END ......svm__C=0.1, svm__gamma=25, svm__kernel=linear; total time=  18.7s
[CV] END ......svm__C=0.1, svm__gamma=25, svm__kernel=linear; total time=  15.9s
[CV] END ......svm__C=5, svm__gamma=0.05, svm__kernel=linear; total time=  15.7s
[CV] END ......svm__C=5, svm__gamma=0.05, svm__kernel=linear; total time=  14.8s
[CV] END .........svm__C=5, svm__gamma=5, svm__kernel=linear; total time=  15.9s
[CV] END .........svm__C=5, svm__gamma=5, svm__kernel=linear; total time=  14.9s
[CV] END ........svm__C=5, svm__gamma=25, svm__kernel=linear; total time=  15.8s
[CV] END ........svm__C=5, svm__gamma=25, svm__ke

In [14]:
best_svm = g_search.best_estimator_
y_pred = best_svm.predict(X_test)

print('Best svm:', best_svm)
print('best svm prediction:', y_pred)

Best svm: Pipeline(steps=[('scaler', StandardScaler()),
                ('svm', SVC(C=5, gamma=0.05, kernel='linear'))])
best svm prediction: [18. 18. 18. ... 11. 18. 11.]


### Neural Net (MLP)

In [26]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier()

In [24]:
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive']
}

grid_search = RandomizedSearchCV(mlp, param_grid, cv=2, scoring='accuracy', n_jobs = 1, verbose =2)
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)


best_mlp = grid_search.best_estimator_
y_pred = best_knn.predict(X_test)

Fitting 2 folds for each of 10 candidates, totalling 20 fits
[CV] END alpha=0.01, hidden_layer_sizes=(50, 50), learning_rate=constant; total time=  19.9s
[CV] END alpha=0.01, hidden_layer_sizes=(50, 50), learning_rate=constant; total time=  35.3s
[CV] END alpha=0.01, hidden_layer_sizes=(50,), learning_rate=constant; total time=  13.4s
[CV] END alpha=0.01, hidden_layer_sizes=(50,), learning_rate=constant; total time=  40.2s
[CV] END alpha=0.001, hidden_layer_sizes=(50,), learning_rate=constant; total time=  15.0s
[CV] END alpha=0.001, hidden_layer_sizes=(50,), learning_rate=constant; total time= 1.5min
[CV] END alpha=0.0001, hidden_layer_sizes=(100,), learning_rate=constant; total time=  40.5s
[CV] END alpha=0.0001, hidden_layer_sizes=(100,), learning_rate=constant; total time=  38.0s
[CV] END alpha=0.001, hidden_layer_sizes=(100,), learning_rate=adaptive; total time=  19.3s
[CV] END alpha=0.001, hidden_layer_sizes=(100,), learning_rate=adaptive; total time= 1.6min
[CV] END alpha=0.001,

## Exercise 2

The model with the best performance was the 
Create an ensemble of at least 25 models, and use them for the classification task. Identify the top and bottom 10% of the data in terms of uncertainty of the decision.

In [15]:
from sklearn.utils import resample

def ensemble(n_models):
    ensemble_preds = []
    for i in range(n_models):
        #boostrap training data with random state
        X_train_bootstrap, y_train_bootstrap = resample(X_train, y_train, replace=True, n_samples=len(X_train), random_state=i)
        #use best model to fit
        best_svm.fit(X_train_bootstrap, y_train_bootstrap)
        pred = best_svm.predict(X_test)
        ensemble_preds.append(pred)
        print('model added')
    return ensemble_preds

ensemble_predictions = ensemble(25)
        

model added
model added
model added
model added
model added
model added
model added
model added
model added
model added
model added
model added
model added
model added
model added
model added
model added
model added
model added
model added
model added
model added
model added
model added
model added


In [17]:
len(ensemble_predictions)

25

In [16]:
import numpy as np
ensemble_preds = np.array(ensemble_predictions)
prediction_variances = np.std(ensemble_preds, axis=0)


top_10_percent_uncertainty = np.argsort(prediction_variances)[-int(0.1 * len(prediction_variances)):]
bottom_10_percent_uncertainty = np.argsort(prediction_variances)[:int(0.1 * len(prediction_variances))]

In [49]:
type(top_10_percent_uncertainty)

numpy.ndarray

## 3. Use 2 different feature selection algorithm to identify the 10 most important features for the task in question 1. Retrain classifiers in question 1 with just this subset of features and compare performance.


In [17]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
#random forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]  # Sort in descending order


top_10_rf = indices[:10]

In [18]:
top_10_rf

array([ 19,   1,  38,  55,  20,  26,  32, 116,  29,  30])

In [ ]:
#RFE
from sklearn.feature_selection import RFE

bestparam = SVC(C=5, gamma=0.05, kernel='linear')
selector = RFE(estimator=bestparam, n_features_to_select=10, step=1)
selector.fit(X_train, y_train)

top_10_rfe = selector.get_support(indices=True)

In [ ]:
top_10_rfe

In [18]:
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

# Subset the data to include only the top 10 features from Random Forest
X_train_rf_top_10 = X_train.iloc[:, top_10_rf]
X_test_rf_top_10 = X_test.iloc[:, top_10_rf]

In [20]:
# Retrain
svm_top_10 = LinearSVC(C=5, max_iter=10000, tol=1e-3)
svm_top_10.fit(X_train_rf_top_10, y_train)
y_pred = svm_top_10.predict(X_test_rf_top_10)
print("SVM Score:", accuracy_score(y_test, y_pred))


SVM Score: 0.9803856080157887


In [21]:
knn_t10 = KNeighborsClassifier(leaf_size=20, n_neighbors=3)
knn_t10.fit(X_train_rf_top_10, y_train)
y_pred = knn_t10.predict(X_test_rf_top_10)
print("KNN Score:", accuracy_score(y_test, y_pred))

KNN Score: 0.9974697636759273


In [29]:
mlp_t10 = MLPClassifier(learning_rate= 'constant', hidden_layer_sizes=(100,), alpha= 0.0001)
mlp_t10.fit(X_train_rf_top_10, y_train)
y_pred = mlp_t10.predict(X_test_rf_top_10)
print("MLP Score:", accuracy_score(y_test, y_pred))

MLP Score: 0.9849197915085269


### With the selected features from the random forest algorithm, the KNN, SVM, and MLP had reduced performance. 

## 4. Use the same data, removing the labels, and compare performance of 3 different clustering algorithms. Can you find clusters for each of the classes in question 1? 

In [17]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score

minibatch_kmeans = MiniBatchKMeans(n_clusters=15, random_state=42, batch_size=500, n_init=3, max_iter=50)
minibatch_kmeans_labels = minibatch_kmeans.fit_predict(X)

In [18]:
centroids = minibatch_kmeans.cluster_centers_
print(minibatch_kmeans_labels)

[11  9  9 ...  9  9  9]


In [19]:
import numpy as np

unique, counts = np.unique(minibatch_kmeans.labels_, return_counts=True)
cluster_counts = dict(zip(unique, counts))
print("Number of points in each cluster:")
print(cluster_counts)

Number of points in each cluster:
{0: 114, 1: 17854, 2: 234587, 3: 34941, 4: 59646, 5: 3598, 6: 4390, 7: 3531, 8: 190, 9: 26791, 10: 945, 11: 9319, 12: 18417, 13: 27632, 14: 52066}


In [19]:
X_sample = X.sample(n=10000, random_state=42)
Xs_t10 = X_sample.iloc[:, top_10_rf]

In [14]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.5, min_samples=5)
labels = dbscan.fit_predict(X_sample)

In [16]:
import numpy as np
unique, counts = np.unique(labels, return_counts=True)
cluster_counts = dict(zip(unique, counts))

print("Number of points in each cluster (including noise):")
print(cluster_counts)

Number of points in each cluster (including noise):
{-1: 4289, 0: 3918, 1: 7, 2: 658, 3: 5, 4: 30, 5: 7, 6: 13, 7: 8, 8: 521, 9: 13, 10: 9, 11: 105, 12: 16, 13: 5, 14: 7, 15: 17, 16: 10, 17: 15, 18: 5, 19: 9, 20: 5, 21: 8, 22: 9, 23: 10, 24: 16, 25: 10, 26: 5, 27: 7, 28: 18, 29: 7, 30: 20, 31: 5, 32: 6, 33: 6, 34: 7, 35: 6, 36: 11, 37: 14, 38: 10, 39: 6, 40: 5, 41: 5, 42: 9, 43: 6, 44: 5, 45: 15, 46: 14, 47: 10, 48: 12, 49: 5, 50: 9, 51: 6, 52: 13, 53: 6, 54: 6, 55: 9, 56: 6, 57: 6}


In [40]:
from sklearn.cluster import AgglomerativeClustering


hierarchical_clustering = AgglomerativeClustering(n_clusters=None, distance_threshold = 10000, linkage="ward")

# Fit the model and predict clusters
labels = hierarchical_clustering.fit_predict(X_sample)


In [42]:
import numpy as np
unique, counts = np.unique(labels, return_counts=True)
cluster_counts = dict(zip(unique, counts))

print("Number of points in each cluster:")
print(cluster_counts)

Number of points in each cluster:
{0: 2705, 1: 27, 2: 40, 3: 8, 4: 16, 5: 280, 6: 92, 7: 14, 8: 12, 9: 57, 10: 95, 11: 2, 12: 59, 13: 2, 14: 7, 15: 59, 16: 24, 17: 53, 18: 133, 19: 138, 20: 7, 21: 1, 22: 401, 23: 1, 24: 46, 25: 1, 26: 3, 27: 1, 28: 1060, 29: 1, 30: 2, 31: 1, 32: 4612, 33: 1, 34: 8, 35: 1, 36: 5, 37: 2, 38: 9, 39: 1, 40: 13}


In [43]:
from sklearn.cluster import MeanShift

mean_shift = MeanShift()
labels = mean_shift.fit_predict(X_sample) 

In [44]:
unique, counts = np.unique(labels, return_counts=True)
cluster_counts = dict(zip(unique, counts))

print("Number of points in each cluster:")
print(cluster_counts)

Number of points in each cluster:
{0: 9785, 1: 46, 2: 17, 3: 89, 4: 8, 5: 8, 6: 8, 7: 11, 8: 5, 9: 6, 10: 2, 11: 2, 12: 2, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1}


## 5. Can you identify any clusters within the top/bottom 10% identified in 2. What are their characteristics?

### Trying reduced dimensionality with chosen features

In [31]:
from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=15)
labels = dbscan.fit_predict(Xs_t10)

In [32]:
unique, counts = np.unique(labels, return_counts=True)
cluster_counts = dict(zip(unique, counts))

print("Number of points in each cluster (including noise, -1):")
print(cluster_counts)

Number of points in each cluster (including noise, -1):
{-1: 4544, 0: 3918, 1: 658, 2: 34, 3: 30, 4: 521, 5: 25, 6: 105, 7: 16, 8: 29, 9: 17, 10: 18, 11: 16, 12: 16, 13: 18, 14: 20, 15: 15}


In [25]:
from sklearn.cluster import AgglomerativeClustering

#hierarchical_clustering = AgglomerativeClustering(n_clusters=None, distance_threshold = 10000, linkage="ward")
# Fit the model and predict clusters
#labels = hierarchical_clustering.fit_predict(Xs_t10)


In [25]:
unique, counts = np.unique(labels, return_counts=True)
cluster_counts = dict(zip(unique, counts))

print("Number of points in each cluster:")
print(cluster_counts)

Number of points in each cluster:
{0: 2, 1: 16, 2: 1683, 3: 214, 4: 2314, 5: 2, 6: 10, 7: 26, 8: 1060, 9: 2, 10: 1, 11: 46, 12: 4616, 13: 1, 14: 6, 15: 1}


In [27]:
from sklearn.cluster import MeanShift

mean_shift = MeanShift()
labels = mean_shift.fit_predict(Xs_t10) 

In [28]:
unique, counts = np.unique(labels, return_counts=True)
cluster_counts = dict(zip(unique, counts))

print("Number of points in each cluster:")
print(cluster_counts)

Number of points in each cluster:
{0: 9898, 1: 46, 2: 11, 3: 20, 4: 9, 5: 6, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1}


### Trying with top and bottom 10%

In [19]:
result_df = pd.DataFrame(X_test)  # Start with the test set data

top_10_percent_uncertainty = top_10_percent_uncertainty.astype(int)
bottom_10_percent_uncertainty = bottom_10_percent_uncertainty.astype(int)

# Add predictions and uncertainty as columns
result_df['ensemble_predictions'] = ensemble_preds.mean(axis=0)  # Average predictions across all models
result_df['prediction_variance'] = prediction_variances  # Add the uncertainty values
    
# Add the top and bottom uncertainty results
result_df['uncertainty'] = 'None'  # Default label
uncertainty_column_pos = result_df.columns.get_loc('uncertainty')


# Label the rows with top 10% uncertainty
result_df.iloc[top_10_percent_uncertainty, uncertainty_column_pos] = 'Top 10% uncertainty'
result_df.iloc[bottom_10_percent_uncertainty, uncertainty_column_pos] = 'Bottom 10% uncertainty'   

In [20]:
result_df.head()
subset_df = result_df[result_df['uncertainty'] != 'None']

In [21]:
subset_df.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_b'RSTR',flag_b'S0',flag_b'S1',flag_b'S2',flag_b'S3',flag_b'SF',flag_b'SH',ensemble_predictions,prediction_variance,uncertainty
317921,0.0,1032.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,18.00,0.000000,Bottom 10% uncertainty
343725,189.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,12.60,1.959592,Top 10% uncertainty
53252,0.0,215.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,10.60,1.897367,Top 10% uncertainty
91622,0.0,6.0,144.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6.60,5.388877,Top 10% uncertainty
140587,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,13.16,3.413854,Top 10% uncertainty


In [22]:
subset_df.iloc[:,:-3]

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_b'REJ',flag_b'RSTO',flag_b'RSTOS0',flag_b'RSTR',flag_b'S0',flag_b'S1',flag_b'S2',flag_b'S3',flag_b'SF',flag_b'SH'
317921,0.0,1032.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
343725,189.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
53252,0.0,215.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
91622,0.0,6.0,144.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
140587,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485178,45.0,2336.0,4201.0,0.0,0.0,0.0,3.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
759,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
41718,15127.0,1229.0,185137.0,0.0,0.0,0.0,0.0,0.0,1.0,767.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
39370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=15)
labels = dbscan.fit_predict(subset_df.iloc[:,:-3])

In [69]:
unique, counts = np.unique(labels, return_counts=True)
cluster_counts = dict(zip(unique, counts))

print("Number of points in each cluster (including noise, -1):")
print(cluster_counts)

Number of points in each cluster (including noise, -1):
{-1: 8673, 0: 7704, 1: 27, 2: 27, 3: 1377, 4: 197, 5: 1046, 6: 19, 7: 22, 8: 37, 9: 24, 10: 22, 11: 17, 12: 21, 13: 20, 14: 17, 15: 55, 16: 32, 17: 21, 18: 17, 19: 26, 20: 15, 21: 31, 22: 22, 23: 15, 24: 28, 25: 15, 26: 27, 27: 36, 28: 28, 29: 29, 30: 23, 31: 22, 32: 22, 33: 16, 34: 15, 35: 15}


In [70]:
hierarchical_clustering = AgglomerativeClustering(n_clusters=None, distance_threshold = 10000, linkage="ward")
# Fit the model and predict clusters
labels = hierarchical_clustering.fit_predict(subset_df.iloc[:,:-3])


In [71]:
unique, counts = np.unique(labels, return_counts=True)
cluster_counts = dict(zip(unique, counts))

print("Number of points in each cluster:")
print(cluster_counts)

Number of points in each cluster:
{0: 33, 1: 4, 2: 14, 3: 1434, 4: 4, 5: 2, 6: 95, 7: 293, 8: 30, 9: 169, 10: 4, 11: 2, 12: 13, 13: 519, 14: 10, 15: 19, 16: 57, 17: 158, 18: 30, 19: 1, 20: 1, 21: 4431, 22: 9, 23: 288, 24: 81, 25: 90, 26: 22, 27: 77, 28: 16, 29: 39, 30: 2194, 31: 5, 32: 1, 33: 9093, 34: 35, 35: 1, 36: 1, 37: 72, 38: 33, 39: 1, 40: 1, 41: 1, 42: 1, 43: 1, 44: 1, 45: 77, 46: 11, 47: 1, 48: 1, 49: 2, 50: 1, 51: 1, 52: 13, 53: 1, 54: 258, 55: 1, 56: 6, 57: 1}


### with top 10 features of the uncertain data

In [23]:
subset = subset_df.iloc[:,:-3]
Xst10_u = subset.iloc[:, top_10_rf]

In [74]:
dbscan = DBSCAN(eps=0.5, min_samples=15)
labels = dbscan.fit_predict(subset_df.iloc[:,:-3])

unique, counts = np.unique(labels, return_counts=True)
cluster_counts = dict(zip(unique, counts))

print("Number of points in each cluster (including noise, -1):")
print(cluster_counts)

Number of points in each cluster (including noise, -1):
{-1: 8673, 0: 7704, 1: 27, 2: 27, 3: 1377, 4: 197, 5: 1046, 6: 19, 7: 22, 8: 37, 9: 24, 10: 22, 11: 17, 12: 21, 13: 20, 14: 17, 15: 55, 16: 32, 17: 21, 18: 17, 19: 26, 20: 15, 21: 31, 22: 22, 23: 15, 24: 28, 25: 15, 26: 27, 27: 36, 28: 28, 29: 29, 30: 23, 31: 22, 32: 22, 33: 16, 34: 15, 35: 15}


In [28]:
hierarchical_clustering = AgglomerativeClustering(n_clusters=None, distance_threshold = 10000, linkage="ward")
# Fit the model and predict clusters
labels = hierarchical_clustering.fit_predict(subset_df.iloc[:,:-3])

unique, counts = np.unique(labels, return_counts=True)
cluster_counts = dict(zip(unique, counts))

print("Number of points in each cluster:")
print(cluster_counts)
subset_df = subset_df.copy()
subset_df['cluster'] = labels

Number of points in each cluster:
{0: 33, 1: 4, 2: 14, 3: 1434, 4: 4, 5: 2, 6: 95, 7: 293, 8: 30, 9: 169, 10: 4, 11: 2, 12: 13, 13: 519, 14: 10, 15: 19, 16: 57, 17: 158, 18: 30, 19: 1, 20: 1, 21: 4431, 22: 9, 23: 288, 24: 81, 25: 90, 26: 22, 27: 77, 28: 16, 29: 39, 30: 2194, 31: 5, 32: 1, 33: 9093, 34: 35, 35: 1, 36: 1, 37: 72, 38: 33, 39: 1, 40: 1, 41: 1, 42: 1, 43: 1, 44: 1, 45: 77, 46: 11, 47: 1, 48: 1, 49: 2, 50: 1, 51: 1, 52: 13, 53: 1, 54: 258, 55: 1, 56: 6, 57: 1}


Big clusters in agglomerative:
3, 7, 13, 21, 23, 30, 33, 54
smaller:
9, 17, 6

In [53]:
c_3 = subset_df.loc[subset_df['cluster'] == 3]
c_21 = subset_df.loc[subset_df['cluster'] == 21]
c_30 = subset_df.loc[subset_df['cluster'] == 30]
c_33 = subset_df.loc[subset_df['cluster'] == 33]
c_33

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_b'S0',flag_b'S1',flag_b'S2',flag_b'S3',flag_b'SF',flag_b'SH',ensemble_predictions,prediction_variance,uncertainty,cluster
317921,0.0,1032.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,18.0,0.0,Bottom 10% uncertainty,33
323343,0.0,1032.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,18.0,0.0,Top 10% uncertainty,33
312866,0.0,1032.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,18.0,0.0,Top 10% uncertainty,33
342570,0.0,1032.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,18.0,0.0,Top 10% uncertainty,33
275412,0.0,1032.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,18.0,0.0,Top 10% uncertainty,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303903,0.0,1032.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,18.0,0.0,Bottom 10% uncertainty,33
209408,0.0,1032.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,18.0,0.0,Bottom 10% uncertainty,33
254759,0.0,1032.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,18.0,0.0,Bottom 10% uncertainty,33
185672,0.0,1032.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,18.0,0.0,Bottom 10% uncertainty,33


In [57]:
c_30['src_bytes'].unique()
c_30['src_bytes'].mode()
c_30["flag_b'SF'"].mode()
c_30['ensemble_predictions'].mode()

0    18.0
Name: ensemble_predictions, dtype: float32

In [55]:
c_33['src_bytes'].unique()
c_33['src_bytes'].mode()
c_33['uncertainty'].mode()
c_33['ensemble_predictions'].unique()


array([18.], dtype=float32)

### The cluster characteristics are not very homogenous, but the ones that are (33, 30) have the same or similar src_bytes values with a 1 for flag_b'SF', and 0.0 for all other values.

# 6. Use the "SA" dataset to compare the performance of 3 different anomaly detection algorithms.

In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
from sklearn.datasets import fetch_kddcup99


# Load and preprocess the SA dataset
data = fetch_kddcup99(subset = 'SA', as_frame = True)
X = pd.DataFrame(data.data)
y = data.target

In [3]:
pt_enc = pd.get_dummies(X['protocol_type'], prefix='ptype', dtype = 'int')
pt_enc.head()
X = pd.concat([X, pt_enc], axis=1)
X.drop('protocol_type', axis=1, inplace=True)

In [4]:
s_enc = pd.get_dummies(X['service'], prefix='service', dtype = 'int')
s_enc.head()
X = pd.concat([X, s_enc], axis=1)
X.drop('service', axis=1, inplace=True)

In [5]:
f_enc = pd.get_dummies(X['flag'], prefix='flag', dtype = 'int')
f_enc.head()
X = pd.concat([X, f_enc], axis=1)
X.drop('flag', axis=1, inplace=True)

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [8]:
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.cluster import MiniBatchKMeans


# Define models
models = {
    "Isolation Forest": IsolationForest(random_state=42),
    "Local Outlier Factor": LocalOutlierFactor(n_neighbors=20, novelty=True),
    "kmeans": MiniBatchKMeans(n_clusters=10, random_state=42)
}

# Fit models and collect results
results = {}
for name, model in models.items():
    print(f"\nRunning {name}...")
    
    # Special handling
    if name == "kmeans":
        labels = model.fit_predict(X_scaled)
        anomaly_scores = -labels  # Outliers are labeled as -1
    elif name == "Local Outlier Factor":
        model.fit(X_scaled)
        anomaly_scores = -model.negative_outlier_factor_
    else:
        model.fit(X_scaled)
        anomaly_scores = model.decision_function(X_scaled)
    print('computing metrics')
    # Compute metrics
    mean_score = anomaly_scores.mean()

    results[name] = {
        "Mean Anomaly Score": mean_score
    }

# Print results
for model, metrics in results.items():
    print(f"\n{model} Results:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")


Running Isolation Forest...
computing metrics

Running Local Outlier Factor...
computing metrics

Running kmeans...
hmm
computing metrics

Isolation Forest Results:
Mean Anomaly Score: 0.1074

Local Outlier Factor Results:
Mean Anomaly Score: 77692.2477

kmeans Results:
Mean Anomaly Score: -6.2427


# 7. Create a subsample of 250 datapoints, redo question 6, using Leave-one-out as the method of evaluation.

In [12]:
from sklearn.utils import resample
from sklearn.model_selection import LeaveOneOut

# Subsample the dataset to 250 points
X_sampled = resample(X_scaled, n_samples=250, random_state=42, replace=False)

In [36]:
loo = LeaveOneOut()
results = {name: {"Precision": [], "Recall": [], "F1-Score": []} for name in models}

for train_index, test_index in loo.split(X_sampled):
    X_train, X_test = X_sampled[train_index], X_sampled[test_index]
    
    true_label = [0]  # Assume the test point is an inlier
    
    for name, model in models.items():
        if name == "Local Outlier Factor":
            model.fit(X_train)
            scores = -model.decision_function(X_test)  # LOF anomaly score
        elif name == "kmeans":
            model.fit(X_train)
            test_distances = model.transform(X_test).min(axis=1)  # Min distance to a centroid
            scores = -test_distances  # Higher distances are more anomalous (negative for consistency)
            threshold = np.percentile(model.transform(X_train).min(axis=1), 95)
            preds = (test_distances > threshold).astype(int)
        else:
            model.fit(X_train)
            scores = model.decision_function(X_test)  # For models like IsolationForest, One-Class SVM
        
        # For other models (non-KMeans), apply the threshold
        if name != "kmeans":
            threshold = np.percentile(scores, 13)  # Threshold for other models
            preds = (scores < threshold).astype(int)  # Lower scores = anomaly
        
        # DEBUG: Print scores and predictions
        #rint(f"\nModel: {name}")
        #print(f"Scores: {scores}")
        #print(f"Predictions: {preds}")
        
        # Compute metrics
        precision = precision_score(true_label, preds, zero_division=0)
        recall = recall_score(true_label, preds, zero_division=0)
        f1 = f1_score(true_label, preds, zero_division=0)
        
        results[name]["Precision"].append(precision)
        results[name]["Recall"].append(recall)
        results[name]["F1-Score"].append(f1)

# Aggregate results
final_results = {model: {metric: np.mean(values) for metric, values in metrics.items()} for model, metrics in results.items()}

# Display results
for model, metrics in final_results.items():
    print(f"\n{model} Results:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")



Isolation Forest Results:
Precision: 0.0000
Recall: 0.0000
F1-Score: 0.0000

Local Outlier Factor Results:
Precision: 0.0000
Recall: 0.0000
F1-Score: 0.0000

kmeans Results:
Precision: 0.0000
Recall: 0.0000
F1-Score: 0.0000


# 8. Use the feature selection algorithm to identify the 5 most important features for the task in question 6, for each algorithm. Does the anomaly detection improve using less features?

In [45]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve, auc
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import StratifiedKFold
import numpy as np

from sklearn.metrics import precision_recall_curve, auc
import numpy as np

y_sampled = resample(y, n_samples=250, random_state=42, replace=False)
# Assuming that `y_test` is binary (0 for inliers, 1 for anomalies)

results = {name: {"Precision-Recall AUC Full": [], "Precision-Recall AUC Selected": []} for name in models}

loo = LeaveOneOut()

# Ensuring y_sampled is binary (0 for normal, 1 for anomaly)
y_sampled = np.array([0 if label == 'normal' else 1 for label in y_sampled])

# Initialize the results dictionary
results = {name: {"Precision-Recall AUC Full": [], "Precision-Recall AUC Selected": []} for name in models}

# Using StratifiedKFold for better handling of the small dataset
loo = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

for train_index, test_index in loo.split(X_sampled, y_sampled):
    X_train, X_test = X_sampled[train_index], X_sampled[test_index]
    y_train, y_test = y_sampled[train_index], y_sampled[test_index]

    # Feature Selection
    X_train_selected = X_train[:, selected_features]
    X_test_selected = X_test[:, selected_features]

    for name, model in models.items():
        # Full Feature Set
        model.fit(X_train)
        scores_full = model.decision_function(X_test) if hasattr(model, 'decision_function') else model.transform(X_test).min(axis=1)
        
        # Calculate Precision-Recall AUC for Full Feature Set
        precision_full, recall_full, _ = precision_recall_curve(y_test, scores_full)
        pr_auc_full = auc(recall_full, precision_full)

        # Selected Feature Set
        model.fit(X_train_selected)
        scores_selected = model.decision_function(X_test_selected) if hasattr(model, 'decision_function') else model.transform(X_test_selected).min(axis=1)
        
        # Calculate Precision-Recall AUC for Selected Feature Set
        precision_selected, recall_selected, _ = precision_recall_curve(y_test, scores_selected)
        pr_auc_selected = auc(recall_selected, precision_selected)

        # Store results
        results[name]["Precision-Recall AUC Full"].append(pr_auc_full)
        results[name]["Precision-Recall AUC Selected"].append(pr_auc_selected)

# Step 3: Average Results
final_results = {model: {metric: np.mean(values) for metric, values in metrics.items()} for model, metrics in results.items()}

# Display Results
for model, metrics in final_results.items():
    print(f"\n{model} Results:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")


Isolation Forest Results:
Precision-Recall AUC Full: 1.0000
Precision-Recall AUC Selected: 1.0000

Local Outlier Factor Results:
Precision-Recall AUC Full: 1.0000
Precision-Recall AUC Selected: 1.0000

kmeans Results:
Precision-Recall AUC Full: 1.0000
Precision-Recall AUC Selected: 1.0000


### Yes, detection improves with less features